# Temporal Leakage

In [ ]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Copy of 2024-06-15_dat_claims.csv', encoding='latin-1')

<ipython-input-1-26d0fc3299ac>:6: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,10,11,12,13,14,15,16,17,18,19,21,25,26,27,28,29,30,31,32,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,51,52,53,54,55,56,57,59,60,61,62,63,64,65,66,67,73,74,75,77,78,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,100,101,102,103,104,105,108,109,110,113,114,116,117,118,119,120) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Copy of 2024-06-15_dat_claims.csv', encoding='latin-1')


In [ ]:
df.claim.nunique()

581414

In [ ]:
len(df.claim[df.claim.notna()])

1006486

In [ ]:
len(df)

1308477

In [ ]:
df.head()

,claim,label,question,answer,positive_evidence_text,positive_evidence_title,negative_evidence_text,negative_evidence_title,split,dataset,...,verifiable,evidence_sent_index,evidence_type,is_auth,evidence_id,evidence_label,article,evidence,entropy,veracity
0,the 76th season was the last time that the bil...,supports,what season was the last time that the bills w...,76th season,NaN,NaN,the last time\r\n\r\n\r\n\r\n\r\n\r\nthe last ...,the last time,validation,faviq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,audie england wrote delta of venus (movie).,refutes,who wrote delta of venus (movie)?,audie england,NaN,NaN,"in life, ""b. daggetti"" is thought to have rese...",buteogallus daggetti,validation,faviq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,new york yankees is the player that has won th...,refutes,who's the player that has won the most world s...,new york yankees,NaN,NaN,world series baseball\r\n\r\n\r\n\r\n\r\n\r\nw...,world series baseball,validation,faviq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,bruce springsteen played the super bowl on the...,supports,what date did bruce springsteen play the super...,"february 1, 2009",springsteen performed at the halftime show at ...,bruce springsteen,"bullet::::- 2003: ""pickin' on bruce springstee...",bruce springsteen discography,validation,faviq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
4,nbc was the company that first aired presentat...,supports,what company first aired presentations of the ...,nbc,section::::original radio series.\r\n\r\n\r\n\...,lone ranger,"an original radio drama of ""kimi ga nozomu eie...",kimi ga nozomu eien,validation,faviq,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [ ]:
import pandas as pd

grouped = df.groupby('dataset')

# Filter out groups where 'data_column' is completely NaN
groups_with_data = grouped.filter(lambda x: x['date'].notna().any())

# Extract distinct categories with data
categories_with_data = groups_with_data['dataset'].unique()

print(categories_with_data)

['fibvid' 'coaid' 'x_fact' 'rumors' 'liar_new' 'fakecovid']


In [ ]:
df_with_date = df[df.dataset.isin(['liar_new', 'fibvid', 'coaid', 'fakecovid', 'x_fact', 'rumors'])]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

for dset in ['liar_new', 'fibvid', 'coaid', 'fakecovid', 'x_fact', 'rumors']:
  print(dset)

  temp_df = df_with_date[df_with_date.dataset == dset]
  print(len(temp_df))
  #temp_df.veracity = temp_df.label.map({'FALSE':2, 'TRUE':1, 'MOSTLY FALSE':2, 'MOSTLY TRUE':1})
  temp_df = temp_df[temp_df.veracity != 3]
  temp_df = temp_df[temp_df.veracity.notna()]
  temp_df['veracity'] = temp_df['veracity'].astype(int)
  print(len(temp_df))
  temp_df = temp_df[temp_df.date.notna()]
  print(len(temp_df))

  temp_df['dates'] = pd.to_datetime(temp_df['date'], format='mixed')

  # Find the earliest (minimum) date in the column
  earliest_date = temp_df['dates'].min()

  # Calculate the time since the earliest date in days
  temp_df['time_since_first'] = ((temp_df['dates'] - earliest_date).dt.total_seconds() / (60 * 60 * 24)).astype(int)

  # Convert the differences to float values
  temp_df['time_since_first'] = temp_df['time_since_first'].astype(float)

  clf = RandomForestClassifier(max_depth=20, random_state=0)#, class_weight='balanced')

  train, test = train_test_split(temp_df, test_size=0.25, random_state=42)



  clf.fit(temp_df.time_since_first.values.reshape(-1,1), temp_df.veracity.values)
  preds = clf.predict(test.time_since_first.values.reshape(-1,1))
  true = test.veracity.values
  print(classification_report(preds,true, digits=3))

liar_new
1957
1957
1957
              precision    recall  f1-score   support

           1      0.083     0.750     0.150         8
           2      0.995     0.863     0.924       482

    accuracy                          0.861       490
   macro avg      0.539     0.807     0.537       490
weighted avg      0.980     0.861     0.912       490

fibvid
2832
2832
1774
              precision    recall  f1-score   support

           1      0.223     0.852     0.354        27
           2      0.988     0.808     0.889       417

    accuracy                          0.811       444
   macro avg      0.606     0.830     0.622       444
weighted avg      0.942     0.811     0.857       444

coaid
278793
278793
278793


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1      1.000     0.936     0.967     69699
           2      0.000     0.000     0.000         0

    accuracy                          0.936     69699
   macro avg      0.500     0.468     0.483     69699
weighted avg      1.000     0.936     0.967     69699

fakecovid
7623
7000
7000
              precision    recall  f1-score   support

           2      1.000     1.000     1.000      1750

    accuracy                          1.000      1750
   macro avg      1.000     1.000     1.000      1750
weighted avg      1.000     1.000     1.000      1750

x_fact
37734
33920
15079
              precision    recall  f1-score   support

           1      0.296     0.683     0.413       461
           2      0.946     0.774     0.851      3309

    accuracy                          0.763      3770
   macro avg      0.621     0.728     0.632      3770
weighted avg      0.867     0.763     0.798      3770

rumors
1236
850
850
   

In [ ]:
# Filter out groups where 'data_column' is completely NaN
groups_with_data = grouped.filter(lambda x: x['tweet_id'].notna().any())

# Extract distinct categories with data
categories_with_data = groups_with_data['dataset'].unique()

print(categories_with_data)

['covid_vaccine_misinfo_mic' 'mediaeval' 'cmu_miscov19' 'antivax'
 'covid-19-disinformation' 'coaid' 'twitter15' 'twitter16'
 'counter-covid-19-misinformation' 'wico']


In [ ]:
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

for dset in ['mediaeval', 'cmu_miscov19', 'antivax',
 'covid-19-disinformation', 'coaid', 'twitter15', 'twitter16',
 'counter-covid-19-misinformation', 'wico']: # 'covid_vaccine_misinfo_mic' has no label?
  print(dset)

  temp_df = df[df.dataset == dset]
  print(len(temp_df))
  temp_df = temp_df[temp_df.veracity != 3]
  temp_df = temp_df[temp_df.veracity.notna()]
  temp_df['veracity'] = temp_df['veracity'].astype(int)
  print(len(temp_df))
  temp_df = temp_df[temp_df.tweet_id.notna()]
  print(len(temp_df))

  def convert_to_int(x):
    x = str(x)[:4]
    if '.' in x:
      x = x.replace('.','')
    else:
      x = x[:3]

    try:
      return int(x)
    except ValueError:
      return 'ERROR'

  temp_df['dates'] = temp_df['tweet_id'].apply(convert_to_int)
  print(len(temp_df[temp_df.dates == 'ERROR']))
  temp_df = temp_df[temp_df.dates != 'ERROR']
  temp_df['dates'] = temp_df['dates'].astype(int)

  clf = RandomForestClassifier(max_depth=20, random_state=0) #, class_weight='balanced')

  train, test = train_test_split(temp_df, test_size=0.25, random_state=42)

  clf.fit(temp_df.dates.values.reshape(-1,1), temp_df.veracity.values)
  preds = clf.predict(test.dates.values.reshape(-1,1))
  true = test.veracity.values
  print(classification_report(preds,true, digits=3))

mediaeval
41948
40366
40323
1
              precision    recall  f1-score   support

           1      0.775     0.677     0.723      5389
           2      0.676     0.774     0.722      4692

    accuracy                          0.722     10081
   macro avg      0.726     0.726     0.722     10081
weighted avg      0.729     0.722     0.722     10081

cmu_miscov19
4573
1761
1761
0
              precision    recall  f1-score   support

           1      0.000     0.000     0.000         0
           2      1.000     0.837     0.911       441

    accuracy                          0.837       441
   macro avg      0.500     0.418     0.456       441
weighted avg      1.000     0.837     0.911       441

antivax


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


15073
15073
15073
0
              precision    recall  f1-score   support

           1      0.100     0.644     0.173       225
           2      0.965     0.631     0.764      3544

    accuracy                          0.632      3769
   macro avg      0.533     0.638     0.468      3769
weighted avg      0.914     0.632     0.728      3769

covid-19-disinformation
63480
19166
19166
0
              precision    recall  f1-score   support

           1      1.000     0.865     0.927      4790
           2      0.002     0.500     0.003         2

    accuracy                          0.865      4792
   macro avg      0.501     0.682     0.465      4792
weighted avg      0.999     0.865     0.927      4792

coaid
278793
278793
275732
0


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           1      1.000     0.938     0.968     68933
           2      0.000     0.000     0.000         0

    accuracy                          0.938     68933
   macro avg      0.500     0.469     0.484     68933
weighted avg      1.000     0.938     0.968     68933

twitter15
1490
1116
1116
0
              precision    recall  f1-score   support

           1      0.690     0.845     0.759        71
           2      0.943     0.870     0.905       208

    accuracy                          0.864       279
   macro avg      0.816     0.858     0.832       279
weighted avg      0.878     0.864     0.868       279

twitter16
818
613
613
0
              precision    recall  f1-score   support

           1      0.932     0.982     0.957        56
           2      0.989     0.959     0.974        98

    accuracy                          0.968       154
   macro avg      0.961     0.971     0.965       154
weighted avg      0.969

<ipython-input-17-cb1525422d57>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df['veracity'] = temp_df['veracity'].astype(int)


156841
1705
1705
0
              precision    recall  f1-score   support

           2      1.000     1.000     1.000       427

    accuracy                          1.000       427
   macro avg      1.000     1.000     1.000       427
weighted avg      1.000     1.000     1.000       427

wico
9685
9685
9685
0
              precision    recall  f1-score   support

           1      1.000     0.677     0.807      2419
           2      0.004     1.000     0.008         3

    accuracy                          0.677      2422
   macro avg      0.502     0.838     0.407      2422
weighted avg      0.999     0.677     0.806      2422

